In [2]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# -------------------------------
# Device Configuration
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------
# Paths and Checkpoints
# -------------------------------
best_model_path = "./models/cifar10_best_student_model.pth"  # Path to best model checkpoint

# -------------------------------
# Evaluation Transform and Test Dataset
# -------------------------------
norm = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    norm
])

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=eval_transform)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

# -------------------------------
# Prepare the Student Model
# -------------------------------
# Using SqueezeNet1_1 as the student model, adapted for CIFAR-10.
student = torchvision.models.squeezenet1_1(pretrained=False)
# Modify the classifier to output 10 classes
student.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
student.num_classes = 10
student.to(device)

# Load the checkpoint for the best student model.
if os.path.exists(best_model_path):
    student.load_state_dict(torch.load(best_model_path, map_location=device))
    print("Loaded best student model from", best_model_path)
else:
    print("Error: Best model checkpoint not found at", best_model_path)
    exit(1)

student.eval()  # Set model to evaluation mode

# -------------------------------
# Define Loss Function (for evaluation)
# -------------------------------
criterion = nn.CrossEntropyLoss()

# -------------------------------
# Evaluate the Model on the Test Set
# -------------------------------
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = student(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        
        # Compute predictions
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

avg_loss = test_loss / total
accuracy = 100. * correct / total

print("Test Loss: {:.4f}".format(avg_loss))
print("Test Accuracy: {:.2f}%".format(accuracy))


Files already downloaded and verified


KeyboardInterrupt: 